# This script runs a prototype feed-forward prototype of the neural network that will be run on a zybo fpga. The script will produce a c-compatible file with values representations stored in (hex??). 

### Todo
- [ ] convert decimal to binary.
- [ ] create a binary multiplier function (optimal?)
- [ ] choose a prototype architecture and store relevant results in a certain format.
- [ ] Decide how software/hardware fixed-point results will be compared to verify correct implementation on hardware? Do i need to do it in C on the Zynq?

### Outstanding Questions

- How should I implement the multiplier in python?

## How to choose an architecture?
my bit representation is S4.27 -> meaning I have a two's complement sign bit, 4 integer bits and 27 fractional bits.

What does this mean?
- fractional resolution is: 0.00000000372529029846
    - I can represent 7 fractional digits accurately?
Biggest magnitude number I can represent is +/- (16 - 0.00000000372529029846) Right?
This is probably overkill. Would be worthwhile looking at reviews of fixed point precision compared to various metrics.

I will stick with this convention for this milestone, but can try different conventions for future milestones.

In [57]:
for entry in os.listdir(os.path.join(os.getcwd(),"bin_files")):
    # print(entry)
    pass

In [56]:
import numpy as np
from fxpmath import Fxp
import sys
import os
import glob

In [75]:
%run plnn_validation.py

['0000000001101111000101110100111011011010111000011001011110001110', '1111111111000101100010011010011111001001111011010101110000101001', '0000000000111100111110101111111010100101111000110101010111100010']
['1111111111001101111101100011000001001011111001111110011111011000']
Success!


In [59]:
%run create_img_h_file.py

FileNotFoundError: [Errno 2] No such file or directory: 'C:/git_repos/mnist_neuralnet/scripts/MNIST_single_test_img1.txt'

# Workbench

In [1]:
import numpy as np
from fxpmath import Fxp

In [2]:
# Thank you Chatgpt!!!
def twos_complement(bin_str):
    """Returns the two's complement of a binary string."""
    # Invert the bits
    inverted = ''.join('1' if bit == '0' else '0' for bit in bin_str)
    # Add 1 to the inverted binary string
    carry = 1
    result = list(inverted)
    
    for i in range(len(inverted) - 1, -1, -1):
        if result[i] == '1' and carry == 1:
            result[i] = '0'
        elif result[i] == '0' and carry == 1:
            result[i] = '1'
            carry = 0
        # No carry to add; break early
        if carry == 0:
            break
    
    return ''.join(result)

def add_binary(bin_str1, bin_str2):
    """Adds two binary strings and returns the result as a binary string, 
    handling overflow by setting the result to the max/min value depending on overflow direction."""
    max_len = max(len(bin_str1), len(bin_str2))
    bin_str1 = bin_str1.zfill(max_len)
    bin_str2 = bin_str2.zfill(max_len)
    
    carry = 0
    result = []
    
    # Perform bitwise addition from LSB to MSB
    for i in range(max_len - 1, -1, -1):
        bit_sum = carry + int(bin_str1[i]) + int(bin_str2[i])
        result.append(str(bit_sum % 2))
        carry = bit_sum // 2
    
    result = ''.join(result[::-1])  # Reverse the result to get the correct order
    
    # Check for overflow
    sign1 = bin_str1[0]  # Sign bit of the first number
    sign2 = bin_str2[0]  # Sign bit of the second number
    result_sign = result[0]  # Sign bit of the result

    # Overflow occurs if both numbers have the same sign but the result has a different sign
    if sign1 == sign2 and sign1 != result_sign:
        if sign1 == '0':  # Positive overflow
            result = '0' + '1' * (max_len - 1)  # Max positive value: 011...111
        else:  # Negative overflow
            result = '1' + '0' * (max_len - 1)  # Max negative value: 100...000
    
    return result


def manual_binary_multiply(bin_str1, bin_str2):
    """Multiplies two binary strings in two's complement format manually."""
    # Check for sign and convert to positive if necessary
    is_negative1 = bin_str1[0] == '1'
    is_negative2 = bin_str2[0] == '1'
    
    if is_negative1:
        bin_str1 = twos_complement(bin_str1)
    if is_negative2:
        bin_str2 = twos_complement(bin_str2)

    # Perform binary multiplication (manual)
    len1 = len(bin_str1)
    len2 = len(bin_str2)
    result = '0' * (len1 + len2)
    
    for i in range(len2 - 1, -1, -1):
        if bin_str2[i] == '1':
            # Shift bin_str1 by (len2 - 1 - i) and add to the result
            shifted_bin_str1 = bin_str1 + '0' * (len2 - 1 - i)
            result = add_binary(result, shifted_bin_str1.zfill(len1 + len2))
    
    # Determine the sign of the result
    if is_negative1 != is_negative2:
        result = twos_complement(result.zfill(len1 + len2))

    # Truncate the result to the appropriate length (len1 + len2 bits)
    return result[-(len1 + len2):]

# Example usage:
bin_str1 = "1101101011010100010110101"  # -3 in 4-bit two's complement
bin_str2 = "1010010111010101010101001"  # -6 in 4-bit two's complement

result = manual_binary_multiply(bin_str1, bin_str2)
print(result)  # 8-bit result in two's complement binary format


00001101000101111000101000111110111001000101111101


In [3]:
bin_str1 = "11011"  # -3 in 4-bit two's complement
bin_str2 = "01101"  # -6 in 4-bit two's complement

add_binary(bin_str1, bin_str2)

'01000'

In [10]:
# weights 784x3
# bias    1x3
# img     1x784
# a =  ( img x weights ) + bias
# a       1x3
x32 = Fxp(-7.25, dtype='S5.27')
x64 = Fxp(-7.25, dtype='S10.54')
len(x32.bin()), len(x64.bin())

num_perceptrons = 3
input_size = 784

weights = (np.random.rand(input_size,num_perceptrons)*2) - 1
img = (np.random.rand(1,input_size) * 2) -1
bias = (np.random.rand(num_perceptrons) * 2) -1
result = np.zeros(num_perceptrons)


result = (img @ weights) + bias
result # this is the ground truth

array([[-11.88238413,  -5.4259064 ,   3.66297954]])

In [37]:
bias

array([-0.78356052, -0.65085528,  0.1529161 ])

In [22]:
for i in range(len(img[0])):
    print(x32(weights[i,1]).bin())

00000110100000110101000110010100
11111001000010000101000001110011
00000001101000101100110110011000
00000000100101100110100111000101
00000101001001111111000000101110
00000000011100010110110001000110
00000000101100101011101000000010
11111101000110010001111101011110
11111110100101111110001100001010
11111100011011000010000011001110
11111011111110110110100111111111
00000111011010110001011010000000
00000010101010011010101000110000
00000110010001011011000100110010
11111100101011001001011010001100
00000011001101100010010100110000
11111101110000001100001110100100
11111000001111111110000010000101
11111111001000101001010111001111
11111110100011011111001101000110
00000010000010101110010001001111
00000011000000010111100010100100
11111100000001000100000101011001
11111001100110010110001011110000
00000100001001001001001011100110
11111101111010010111000000000011
00000101100100101101010110000001
11111011110101011010001000111001
00000011011101100011001010000010
11111110011001101010100010001010
0000010010

In [21]:
for i in range(num_perceptrons):
    print(x32(bias[i]).bin())

11111001101110110100010010100000
11111010110010110000110001100011
00000001001110010010110000010011


In [24]:
x32('0h1d4dc82c')

fxp-s32/27(3.6629794538021088)

In [27]:
input_size = 784
result = (img @ weights) + bias
result # this is the ground truth


a_tdata = []
for j in range(num_perceptrons):
    b = x64(bias[j]).bin()
    acc = b
    for i in range(input_size):
        w,x  = x32(weights[i,j]).bin(), x32(img[0,i]).bin()
        p = manual_binary_multiply(w,x)
        acc = add_binary(acc,p)

        # floating point calc
        
    
    print(f'{j/num_perceptrons}%',end='\r')
    a_tdata.append(acc)

print([str(x64(f'0b{k}')) for k in a_tdata])
print([str(k) for k in result.squeeze()])

['-11.882383951895978', '-5.425906313096542', '3.662979456797086']
['-11.882384125979387', '-5.425906404133703', '3.6629795375015153']


In [26]:
str(x64('0hfd0787057747eba0')), str(x64('0hfea4bdf373d9762e')), str(x64('0h00ea6e4161fafd75'))

('-11.882383950724323', '-5.425906311860811', '3.662979455646528')

In [29]:
a_tdata[0]

'1111110100000111100001110000010101110110000001011101101111001000'

In [30]:
'1111110100000111100001110000010101110111010001111110101110100000'

'1111110100000111100001110000010101110111010001111110101110100000'

In [19]:
bias

array([-0.78356052, -0.65085528,  0.1529161 ])

In [132]:
for i in range(4):
    # print(x32(weights[i,0]).bin(), x32(img[0,i]).bin())
    # print(twos_complement(x32(weights[i,0]).bin()), twos_complement(x32(img[0,i]).bin()))
    # print(weights[i,0] * img[0,i])
    # print(x64(weights[i,0] * img[0,i]).bin(frac_dot=True))
    # print(manual_binary_multiply(x32(weights[i,0]).bin(),x32(img[0,i]).bin()))

00000011111010001011010101010111 00000010100101001011011001011111
11111100000101110100101010101001 11111101011010110100100110100001
0000000000001010000101101011110001101000100100110010010101001001
11111001111011100000101010110010 11111011011001110100000110110010
00000110000100011111010101001110 00000100100110001011111001001110
0000000000011011111001110000001000001011110110001010000111000100
00000000010001100111000000101110 11111101001010011100100001011011
11111111101110011000111111010010 00000010110101100011011110100101
1111111111111111001110000010111010001110000010111101000001011010
00000011101000110100101100010000 00000110100010010011110111011000
11111100010111001011010011110000 11111001011101101100001000101000
0000000000010111110001110000011010000110001110100010010110000000


In [104]:
multiply(*fitlen(x,y))

13473926

In [93]:
x32 = Fxp(-7.25, dtype='S5.27')
x64 = Fxp(-7.25, dtype='S10.54')
len(x32.bin()), len(x64.bin())

(32, 64)

In [ ]:
0.15763769352323159234

In [38]:
for j in range(num_perceptrons):
    with open(f"weight{j}.txt", "r") as file:
        lines = file.readlines()
        # for i in range(input_size):
        #     weights[i, j] = x32(int(lines[i].strip(), 2)).to_float()

In [44]:
int(lines[0].strip())

110101100001000100000000011